In [ ]:
def check_plot(theta,x,sim):
    
    a1,b1, c1, d1,e1,f1,a2,b2,c2, d2,e2,f2, a3,b3,c3,d3,e3,f3, a4,b4,c4,d4,e4,f4, a5,b5,c5,d5,e5,f5 = theta

    emitter_doping = a1*np.log(-1/x) + b1*x + c1*logP + d1*P + e1*x*logP + f1
    back_doping    = a2*np.log(-1/x) + b2*x + c2*logP + d2*P + e2*x*logP + f2
    tau            = a3*np.log(-1/x) + b3*x + c3*logP + d3*P + e3*x*logP + f3
    fsrv           = a4*np.log(-1/x) + b4*x + c4*logP + d4*P + e4*x*logP + f4
    rsrv           = a5*np.log(-1/x) + b5*x + c5*logP + d5*P + e5*x*logP + f5      
    
    par_input = 10*np.stack((emitter_doping,back_doping,tau,fsrv,rsrv),axis=-1)
    if sim == 0 :
        unnorm_par = scaler.inverse_transform(par_input)
        return par_input,unnorm_par
        
    sim_curves= reg.predict(par_input)
           
  
    return sim_curves, par_input

In [ ]:
def log_probability(theta,x,y,sigma):
    a1,b1, c1, d1,e1,f1,a2,b2,c2, d2,e2,f2, a3,b3,c3,d3,e3,f3, a4,b4,c4,d4,e4,f4, a5,b5,c5,d5,e5,f5 = theta
    emitter_doping = a1*np.log(-1/x) + b1*x + c1*np.log(P) + d1*P + e1*x*np.log(P) + f1
    back_doping    = a2*np.log(-1/x) + b2*x + c2*np.log(P) + d2*P + e2*x*np.log(P) + f2
    tau            = a3*np.log(-1/x) + b3*x + c3*np.log(P) + d3*P + e3*x*np.log(P) + f3
    fsrv           = a4*np.log(-1/x) + b4*x + c4*np.log(P) + d4*P + e4*x*np.log(P) + f4
    rsrv           = a5*np.log(-1/x) + b5*x + c5*np.log(P) + d5*P + e5*x*np.log(P) + f5    

    par_input = 10*np.stack((emitter_doping,back_doping,tau,fsrv,rsrv),axis=-1)
    coeff = [a1,b1, c1, d1,e1,f1,a2,b2,c2, d2,e2,f2, a3,b3,c3,d3,e3,f3, a4,b4,c4,d4,e4,f4, a5,b5,c5,d5,e5,f5]

    if all(-10<x<10 for x in coeff) and max(np.abs(coeff[0::3]))<5:
    
        if np.max(par_input)<1 and np.min(par_input)>0:
            sim_curves= reg.predict(par_input)       
        
            return log_norm_pdf(sim_curves, y,sigma)
        return -np.inf
    return -np.inf

In [ ]:
ndim = 30         # 30 parameters: a, b, c,d,e,f × 5 descriptors
nwalkers = 60       # Number of walkers
nruns = 1500    # Number of steps
sigma = 1e-6        # Noise level
Temp_i = 0          # Just indexing; no need for ntemps here

initial = np.tile([0, 0, 0, 0,0,0], 5)/10 + 1e-4*np.random.randn(nwalkers, ndim)

sampler = emcee.EnsembleSampler(nwalkers, ndim, log_prob)

sampler.run_mcmc(initial, nruns, progress=True)
samples = sampler.get_chain()             # Shape: (nruns, nwalkers, ndim)
#flat_samples = sampler.get_chain(discard=100, thin=10, flat=True)  # Flattened, thinned